In [22]:
import requests
import re
from bs4 import BeautifulSoup
import pymorphy2
import datetime
import math
from collections import Counter

In [15]:
requests.get("http://lenta.ru/")

<Response [200]>

In [27]:
findheaders = re.compile('<h1.+?>(.+)</h1>', re.S)
boa = re.compile('div class="b-text clearfix js-topic__text" itemprop="articleBody"><p>', re.S)
eoa = re.compile('<div class="b-box"><i>', re.S)
delscript = re.compile('<script.+?>(.+)</script>', re.S)

In [28]:
def getLentaArticle2(url):
    article = requests.get(url)
    title = findheaders.findall(article.text)[0]
    text = eoa.split( boa.split(article.text)[1] )
    text = "".join(delscript.split(text[0]))
    return BeautifulSoup ( title + '\n--------\n' + text, 'lxml').get_text()

In [35]:
def getLentaArticle(url):
    article = requests.get(url)
    title = findheaders.findall(article.text)[0]
    
    bs = BeautifulSoup ( article.text, 'lxml')
    #title = str(bs('h1'))
    text = delscript.split( str(bs('p'))[1:-1])
    return BeautifulSoup ( title + '\n--------\n' + "".join(text), 'lxml').get_text()

In [50]:
art_text = getLentaArticle('https://lenta.ru/news/2018/09/30/tayga/')
print ( art_text )

Путин отдохнул в тайге и уехал к морю
--------
Президент России Владимир Путин провел субботу, 29 сентября, в тайге. Об этом сообщил пресс-секретарь главы государства Дмитрий Песков, передает РИА Новости., «По дороге из Душанбе Владимир Путин совершил остановку в Хакасии, где провел субботний день в тайге с кратким отдыхом», — рассказал он и добавил, что после этого президент отправился в Сочи., Путин прибыл в столицу Таджикистана для участия в саммите в пятницу, 28 сентября. Там российский лидер, в частности, пообщался с президентом Киргизии Сооронбаем Жээнбековым  и главой Таджикистана Эмомали Рахмоном., В процессе церемонии подписания документов Путин зачитался «Евгением Онегиным». «Сначала Путин положил книгу чуть слева от себя, но, поставив несколько подписей, взял экземпляр стихотворного романа Александра Пушкина и пролистал несколько страниц. Принесли новую порцию документов, президент поставил свою подпись и снова открыл книгу», — писало об этом РИА Новости., В 2017 году Путин 

In [43]:
posConv = {'ADJF':'_ADJ', 'NOUN':'_NOUN', 'VERB':'_VERB'}
meaningfullPoSes = ['ADJF', 'NOUN', 'VERB']

def getArticleDictionary( text, needPos = None):
    words = [a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
    reswords = []
    
    for w in words:
        wordform = morph.parse(w)[0]
        if wordform.tag.POS in meaningfullPoSes:
            if needPos != None:
                reswords.append(wordform.normal_form + posConv[wordform.tag.POS] )
            else:
                reswords.append(wordform.normal_form)
    return Counter ( reswords )

In [70]:
stat1 = getArticleDictionary( art_text, True )
print ( statl )

Counter({'путин_NOUN': 7, 'тайга_NOUN': 4, 'президент_NOUN': 4, 'владимир_NOUN': 2, 'провести_VERB': 2, 'сентябрь_NOUN': 2, 'риа_NOUN': 2, 'новость_NOUN': 2, 'таджикистан_NOUN': 2, 'документ_NOUN': 2, 'книга_NOUN': 2, 'подпись_NOUN': 2, 'отдохнуть_VERB': 1, 'уехать_VERB': 1, 'море_NOUN': 1, 'россия_NOUN': 1, 'суббота_NOUN': 1, 'сообщить_VERB': 1, 'пресс-секретарь_NOUN': 1, 'глава_NOUN': 1, 'государство_NOUN': 1, 'дмитрий_NOUN': 1, 'песок_NOUN': 1, 'передавать_VERB': 1, 'дорога_NOUN': 1, 'душанбе_NOUN': 1, 'совершить_VERB': 1, 'остановка_NOUN': 1, 'хакасия_NOUN': 1, 'субботний_ADJ': 1, 'день_NOUN': 1, 'краткий_ADJ': 1, 'отдых_NOUN': 1, 'рассказать_VERB': 1, 'добавить_VERB': 1, 'отправиться_VERB': 1, 'сочи_NOUN': 1, 'прибыть_VERB': 1, 'столица_NOUN': 1, 'участие_NOUN': 1, 'саммит_NOUN': 1, 'пятница_NOUN': 1, 'российский_ADJ': 1, 'лидер_NOUN': 1, 'частность_NOUN': 1, 'пообщаться_VERB': 1, 'киргизия_NOUN': 1, 'сооронбай_NOUN': 1, 'жээнбеков_NOUN': 1, 'главый_ADJ': 1, 'эмомали_NOUN': 1, 'ра

In [59]:
from sklearn.feature_extraction.text import CountVectorizer
counter = CountVectorizer()
res = counter.fit_transform( [art_text] )
print(counter.get_feature_names() )
print(counter.vocabulary_.get('путин'))

['2017', '28', '29', 'александра', 'взял', 'визита', 'владимир', 'где', 'главой', 'главы', 'году', 'государства', 'день', 'для', 'дмитрий', 'добавил', 'документов', 'дороге', 'душанбе', 'евгением', 'жээнбековым', 'заночевал', 'зачитался', 'из', 'киргизии', 'книгу', 'красноярск', 'кратким', 'лидер', 'морю', 'несколько', 'но', 'новости', 'новую', 'об', 'он', 'онегиным', 'остановку', 'от', 'отдохнул', 'отдыхом', 'открыл', 'отправился', 'передает', 'песков', 'писало', 'по', 'побывал', 'подписания', 'подписей', 'подпись', 'положил', 'пообщался', 'порцию', 'после', 'поставив', 'поставил', 'походных', 'президент', 'президентом', 'пресс', 'прибыл', 'принесли', 'провел', 'пролистал', 'процессе', 'путин', 'пушкина', 'пятницу', 'рабочего', 'рассказал', 'рахмоном', 'риа', 'романа', 'россии', 'российский', 'саммите', 'свою', 'себя', 'секретарь', 'сентября', 'сибирской', 'слева', 'сначала', 'снова', 'совершил', 'сообщил', 'сооронбаем', 'сочи', 'стихотворного', 'столицу', 'страниц', 'субботний', 'суб

In [61]:
counter12 = CountVectorizer(ngram_range=(1,2))
analyse = counter12.build_analyzer()
print( analyse(art_text) )
res= counter12.fit_transform( [art_text] )
print(counter.get_feature_names() )
print(counter.vocabulary_.get('путин'))

['путин', 'отдохнул', 'тайге', 'уехал', 'морю', 'президент', 'россии', 'владимир', 'путин', 'провел', 'субботу', '29', 'сентября', 'тайге', 'об', 'этом', 'сообщил', 'пресс', 'секретарь', 'главы', 'государства', 'дмитрий', 'песков', 'передает', 'риа', 'новости', 'по', 'дороге', 'из', 'душанбе', 'владимир', 'путин', 'совершил', 'остановку', 'хакасии', 'где', 'провел', 'субботний', 'день', 'тайге', 'кратким', 'отдыхом', 'рассказал', 'он', 'добавил', 'что', 'после', 'этого', 'президент', 'отправился', 'сочи', 'путин', 'прибыл', 'столицу', 'таджикистана', 'для', 'участия', 'саммите', 'пятницу', '28', 'сентября', 'там', 'российский', 'лидер', 'частности', 'пообщался', 'президентом', 'киргизии', 'сооронбаем', 'жээнбековым', 'главой', 'таджикистана', 'эмомали', 'рахмоном', 'процессе', 'церемонии', 'подписания', 'документов', 'путин', 'зачитался', 'евгением', 'онегиным', 'сначала', 'путин', 'положил', 'книгу', 'чуть', 'слева', 'от', 'себя', 'но', 'поставив', 'несколько', 'подписей', 'взял', 'эк

In [66]:
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv = morph.parse(t)
        for p in pv:
            if p.tag.POS in meaningfullPoSes:
                words.append(p.normal_form)
                break
    return words



In [67]:
c=[' '.join(getMeaningfullWords(art_text))]
c2=[' '.join([morph.parse(r)[0].normal_form for r in re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', art_text)])]

lemmaCounter=CountVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

analyze = lemmaCounter.build_analyzer()
res1=analyze(c[0])
res2=lemmaCounter.fit_transform(c)
print({w:res2[0][0,lemmaCounter.vocabulary_[w]] for w in lemmaCounter.vocabulary_ if res2[0][0,lemmaCounter.vocabulary_[w]]>1})
print("---")
res1=analyze(c2[0])
res2=lemmaCounter.fit_transform(c2)
print({w:res2[0][0,lemmaCounter.vocabulary_[w]] for w in lemmaCounter.vocabulary_ if res2[0][0,lemmaCounter.vocabulary_[w]]>1})


{'путин': 7, 'век': 14, 'тайга': 4, 'исполняющий': 5, 'президент': 4, 'владимир': 2, 'провести': 2, 'сентябрь': 2, 'этот': 3, 'риа': 2, 'новость': 2, 'секунда': 2, 'посол': 2, 'таджикистан': 2, 'документ': 2, 'книга': 2, 'подпись': 2, 'век тайга': 3, 'владимир путин': 2, 'риа новость': 2}
---
{'путин': 7, 'в': 14, 'тайга': 4, 'и': 5, 'президент': 4, 'владимир': 2, 'провести': 2, 'сентябрь': 2, 'о': 2, 'это': 3, 'риа': 2, 'новость': 2, 'с': 2, 'он': 2, 'после': 2, 'таджикистан': 2, 'документ': 2, 'книга': 2, 'поставить': 2, 'несколько': 2, 'подпись': 2, 'в тайга': 3, 'владимир путин': 2, 'о это': 2, 'риа новость': 2}


In [68]:
def cosineSimilarity(a, b):
    if len(a.keys())==0 or len(b.keys())==0:
        return 0
    sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
    suma2=sum([a[na]*a[na] for na in a.keys()])
    sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
    return sumab/math.sqrt(suma2*sumb2)

In [71]:
stat2=getArticleDictionary(getLentaArticle("https://lenta.ru/news/2018/02/15/pengilly_domoi/"), True)
stat3=getArticleDictionary(getLentaArticle("https://lenta.ru/news/2018/02/15/tar_mor/"), True)
stat4=getArticleDictionary(getLentaArticle("https://lenta.ru/news/2018/02/15/olympmovies/"), True)

print(cosineSimilarity(stat1, stat2))
print(cosineSimilarity(stat1, stat3))
print(cosineSimilarity(stat2, stat3))
print(cosineSimilarity(stat2, stat4))
print(cosineSimilarity(stat3, stat4))

0.009597211839133202
0.051216617822002236
0.12321187388436787
0.10845193904480363
0.08038418992031009


In [73]:
class getNewsPaper:
    articles=[]     # Загруженные статьи.
    dictionaries=[] # Посчитанные словари (векторное представление статей).
        
    # Конструктор - компилирует регулярные выражения и загружает морфологию.
    def __init__(self):
        self.delscript = re.compile("<script.*?>.+?</script>", re.S)
        self.findheaders = re.compile("<h1.+?>(.+)</h1>", re.S)
        self.boa = re.compile('<div class="b-text clearfix js-topic__text" itemprop="articleBody">', re.S)
        self.eoa = re.compile('<div class="b-box">\s*?<i>', re.S)
        self.findURLs = re.compile('<h3>(.+?)</h3>', re.S)
        self.rboa = re.compile('<p class="MegaArticleBody_first-p_2htdt">', re.S)
        self.reoa = re.compile('<div class="Attribution_container_28wm1">', re.S)
        self.rfindURLs = re.compile('''<div class="headlineMed"><a href='(.+?)'>''', re.S)
        # Создаем и загружаем морфологический словарь.
        self.morph=pymorphy2.MorphAnalyzer()

    # Загрузка статьи по URL.
    def getLentaArticle(self, url):
        """ getLentaArticle gets the body of an article from Lenta.ru"""
        # Получает текст страницы.
        art=requests.get(url)
        # Находим заголовок.
        title = findheaders.findall(art.text)[0]
        # Выделяем текст новости.
        bs=BeautifulSoup(art.text, "lxml")
        # Выкусываем скрипты - BeautifulSoup не справляетсяя с ними. bs['p'] выделяет параграфы текста.
        # [1:-1] применяется ля того, чтобы выкусить квадратные скобки, которые добавляются здесь при преобразовании к строке.
        text=delscript.split(str(bs('p'))[1:-1])
        # Выкусываем остальные теги.
        self.articles.append(BeautifulSoup(title+"\n-----\n"+" ".join(text), "lxml").get_text())

    # Загрузка всех статей за один день.
    def getLentaDay(self, url):
        """ Gets all URLs for a given day and gets all texts. """
        try:
            day = requests.get(url) # Грузим страницу со списком всех статей.
            cand = self.findURLs.findall(day.text) # Выделяем адреса статей.
            links = ['https://lenta.ru'+re.findall('"(.+?)"', x)[0] for x in cand]
            for l in links: # Загружаем статьи.
                self.getLentaArticle(l)
        except:
            pass

    # Загрузка всех статей за несколько дней.
    def getLentaPeriod(self, start, finish):
        curdate=start
        while curdate<=finish:
            print(curdate.strftime('%Y/%m/%d')) # Just in case.
            # Список статей грузится с вот такого адреса.
            res=self.getLentaDay('https://lenta.ru/news/'+curdate.strftime('%Y/%m/%d'))
            curdate+=datetime.timedelta(days=1)
      
    # Потроение вектора для статьи.
    posConv={'ADJF':'_ADJ','NOUN':'_NOUN','VERB':'_VERB'}
    def getArticleDictionary(self, text, needPos=None):
        words=[a[0] for a in re.findall("([А-ЯЁа-яё]+(-[А-ЯЁа-яё]+)*)", text)]
        reswords=[]
    
        for w in words:
            wordform=self.morph.parse(w)[0]
            try:
                if wordform.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                    if needPos!=None:
                        reswords.append(wordform.normal_form+self.posConv[wordform.tag.POS])
                    else:
                        reswords.append(wordform.normal_form)
            except:
                pass
            
        stat=Counter(reswords)
#        stat={a: stat[a] for a in stat.keys() if stat[a]>1}
        return stat

    # Посчитаем вектора для всех статей.
    def calcArticleDictionaries(self, needPos=None):
        self.dictionaries=[]
        for a in self.articles:
            self.dictionaries.append(self.getArticleDictionary(a, needPos))

    # Сохраняем стстьи в файл.
    def saveArticles(self, filename):
        """ Saves all articles to a file with a filename. """
        newsfile=open(filename, "w")
        for art in self.articles:
            newsfile.write('\n=====\n'+art)
        newsfile.close()

    # Читаем статьи из файла.
    def loadArticles(self, filename):
        """ Loads and replaces all articles from a file with a filename. """
        newsfile=open(filename, encoding="utf-8")
        text=newsfile.read()
        self.articles=text.split('\n=====\n')[1:]
#        self.articles=[a.replace('\xa0', ' ') for a in text.split('\n=====\n')[1:]]
        newsfile.close()

    # Для удобства - поиск статьи по ее заголовку.
    def findNewsByTitle(self, title):
        for i, a in enumerate(self.articles):
            if title==a.split('\n-----\n')[0]:
                return i
        return -1

def cosineSimilarity(a, b):
    if len(a.keys())==0 or len(b.keys())==0:
        return 0
    sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
    suma2=sum([a[na]*a[na] for na in a.keys()])
    sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
    return sumab/math.sqrt(suma2*sumb2)



In [74]:
# Загрузка статей за заданный период.
# !!! Это рабоатет довольно долго, пользуйтесь сохраненными данными!!!
lenta=getNewsPaper()
# lenta.getLentaPeriod(datetime.date(2018, 2, 1), datetime.date(2018, 2, 14))
# lenta.saveArticles("lenta2018.txt")
lenta.loadArticles("data\lenta2018.txt")
lenta.calcArticleDictionaries()

In [75]:
i1 = 0
maxCos, maxpos = -1, -1
for i in range(len(lenta.articles)):
    if i != i1:
        c = cosineSimilarity(lenta.dictionaries[i1], lenta.dictionaries[i])
        if c>maxCos:
            maxCos, maxpos = c, i
print(lenta.articles[i1].split('\n-----\n')[0])
print(lenta.articles[maxpos].split('\n-----\n')[0])
print(maxCos, maxpos)

Раскрыто происхождение новейшей украинской крылатой ракеты
Россия поставила Украине оружие «сдерживания агрессора»
0.6522716186492549 516


In [76]:
likesport=['Власти США обвинили МОК и ФИФА в коррупции', 'Пробирки WADA для допинг-проб оказались бракованными', 'Пожизненно отстраненных российских спортсменов оправдали', 'В Кремле порадовались за оправданных российских спортсменов', 'Россия вернется на первое место Олимпиады-2014', 'МОК разочаровало оправдание российских олимпийцев', 'Мутко загрустил после оправдания российских спортсменов', 'Оправданный призер Сочи-2014 призвал «добить ситуацию» с МОК', 'Путин предостерег от эйфории после оправдания российских олимпийцев', 'Родченков не смог вразумительно ответить на вопросы суда', 'Оправданный россиянин позлорадствовал над делившими медали Игр-2014 иностранцами', 'В CAS отказались считать оправданных россиян невиновными', 'Адвокат Родченкова заговорил о смерти чистого спорта после оправдания россиян', 'Американская скелетонистка сочла россиян ушедшими от законного наказания']
likeelect=['Социологи подсчитали планирующих проголосовать на выборах-2018', 'Собчак пообещала дать Трампу пару советов', 'На выборы президента России пойдут почти 80 процентов избирателей', 'Песков вспомнил предупреждение и отказался комментировать поездку Собчак в США', 'Собчак съездила на завтрак с Трампом и разочаровалась', 'Грудинин уступил в популярности КПРФ', 'Собчак потребовала признать незаконной регистрацию Путина на выборах', 'У Грудинина обнаружили два не до конца закрытых счета в Швейцарии и Австрии', 'Грудинин раскрыл историю происхождения дома в Испании', 'Путина зарегистрировали кандидатом в президенты', 'В Кремле отреагировали на слухи о голосовании Путина в Севастополе', 'Коммунистов вновь обвинили в незаконной агитации за Грудинина', 'ЦИК выявила обман со стороны Грудинина', 'Грудинин ответил на претензии ЦИК', 'Жириновский захотел сбросить ядерную бомбу на резиденцию Порошенко']

In [77]:
sporttext=' '.join([lenta.articles[lenta.findNewsByTitle(art)] for art in likesport])
sportdict=lenta.getArticleDictionary(sporttext)
electtext=' '.join([lenta.articles[lenta.findNewsByTitle(art)] for art in likeelect])
electdict=lenta.getArticleDictionary(electtext)
#print(sportdict)
#print(electdict)

In [78]:
thrs=0.4
thre=0.5
cosess=[lenta.articles[i].split('\n-----\n')[0] for i in range(len(lenta.dictionaries)) if cosineSimilarity(sportdict, lenta.dictionaries[i])>thrs]
print(cosess)
cosese=[lenta.articles[i].split('\n-----\n')[0] for i in range(len(lenta.dictionaries)) if cosineSimilarity(electdict, lenta.dictionaries[i])>thre]
print(cosese)

['Пожизненно отстраненных российских спортсменов оправдали', 'В Кремле порадовались за оправданных российских спортсменов', 'Россия вернется на первое место Олимпиады-2014', 'МОК разочаровало оправдание российских олимпийцев', 'Мутко загрустил после оправдания российских спортсменов', 'Олимпиада в Пхенчхане побила рекорд по презервативам', 'Оправданный призер Сочи-2014 призвал «добить ситуацию» с МОК', 'Путин предостерег от эйфории после оправдания российских олимпийцев', 'Родченков не смог вразумительно ответить на вопросы суда', 'Оправданный россиянин позлорадствовал над делившими медали Игр-2014 иностранцами', 'В CAS отказались считать оправданных россиян невиновными', 'Адвокат Родченкова заговорил о смерти чистого спорта после оправдания россиян', 'Американская скелетонистка сочла россиян ушедшими от законного наказания', 'Глава USADA почуял вонь российской атаки на чистый спорт', 'После оправдания российских спортсменов Макларена назвали идиотом', 'МОК посчитал оправдание российск

In [79]:
lenta_new=getNewsPaper()
#lenta_new.getLentaPeriod(datetime.date(2018, 2, 15), datetime.date(2018, 2, 15))
#lenta_new.saveArticles("lenta20180215.txt")
lenta_new.loadArticles("data\lenta20180215.txt")
lenta_new.calcArticleDictionaries()

In [80]:
thrs_new = 0.3
thre_new = 0.3
cosess_new = [lenta_new.articles[i].split('\n-----\n')[0] for i in range(len(lenta_new.dictionaries)) if cosineSimilarity(sportdict, lenta_new.dictionaries[i])>thrs_new]
print(cosess_new)
cosese_new = [lenta_new.articles[i].split('\n-----\n')[0] for i in range(len(lenta_new.dictionaries)) if cosineSimilarity(electdict, lenta_new.dictionaries[i])>thre_new]
print(cosese_new)

['Российский сноубордист сломал ногу на Олимпиаде', 'Российскую лыжницу затравили на Олимпиаде']
['Кремль опроверг информацию о засекречивании данных по россиянам в Сирии', 'Порошенко раскрыл детали разговора с Путиным', 'Грудинин оговорился о финансовых махинациях', 'Прогнозируемую явку на выборах президента сочли высокой']


In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [82]:
c=[' '.join(getMeaningfullWords(n)) for n in lenta.articles]

tfCounter=TfidfVectorizer(ngram_range=(1,2), token_pattern=r'[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+')

analyze = tfCounter.build_analyzer()
res=tfCounter.fit_transform(c)

In [83]:
id_article=6

res2=analyze(lenta.articles[id_article])
tfs=list(set(res[id_article][0, tfCounter.vocabulary_.get(k)] for k in res2 if k in tfCounter.vocabulary_.keys()))
tfs2=[k for k in tfs if k>np.average(tfs)]
#tfs2=[k for k in tfs if k>np.average(tfs)+np.std(tfs)]
print({w:res[id_article][0, tfCounter.vocabulary_[w]] for w in res2 if w in tfCounter.vocabulary_.keys() and res[id_article][0, tfCounter.vocabulary_[w]] in tfs2})


{'ближний': 0.09754172040483444, 'восток': 0.10389546512876865, 'платформа': 0.11428148479237515, 'жозе': 0.16535511285226273, 'увеличение': 0.0842033272150634, 'возможность': 0.08574845736217529, 'доставка': 0.13234427079429065, 'клиентский': 0.1744831264097395, 'сервис': 0.0709906385519639, 'услуга': 0.06742359087070612, 'ближний восток': 0.13825073355807999, 'клиентский сервис': 0.11632208427315967}


In [84]:
# Импортируем библиотеки Word2Vec
from gensim.models.word2vec import Word2Vec # Собственно модель.
from gensim.models.word2vec import LineSentence # Выравнивание текста по предложениям.
from gensim.models import KeyedVectors # Семантические вектора.
# На самом деле, нам потребуется только последняя.
import numpy as np # Вектора.

d:\IDE\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [87]:
model = KeyedVectors.load_word2vec_format('data/news_upos_cbow_600_2_2018.vec')

In [88]:
model['огонь_NOUN']

array([ 5.58170e-02,  4.41830e-02, -1.71040e-02, -4.57520e-02,
        2.04150e-02, -2.40040e-02,  4.09340e-02,  3.11130e-02,
       -3.95260e-02,  2.20970e-02,  2.24920e-02,  1.92750e-02,
        2.78080e-02,  4.49070e-02,  5.63220e-02,  4.45900e-03,
        4.61480e-02,  3.07340e-02, -6.30660e-02,  7.29010e-02,
        7.12300e-03, -1.71800e-02,  1.96740e-02,  2.98010e-02,
        2.71750e-02,  8.77070e-02, -1.97520e-02,  3.35100e-03,
       -7.84200e-03,  8.84600e-03, -3.89400e-02,  1.03100e-02,
        2.48070e-02, -4.09000e-03, -1.73660e-02, -7.46190e-02,
        1.10265e-01,  2.89020e-02, -5.00060e-02,  4.90970e-02,
       -4.85360e-02,  3.00110e-02,  3.67230e-02,  3.55780e-02,
       -7.86430e-02, -6.60400e-03, -1.02970e-02, -4.44200e-03,
        6.34600e-02,  1.40280e-02, -6.75700e-03, -1.19300e-03,
       -4.25510e-02, -2.17790e-02,  7.87590e-02, -8.13890e-02,
        1.08440e-02, -3.11530e-02,  6.01860e-02,  8.62800e-03,
       -3.41890e-02, -3.25180e-02, -1.70000e-05, -1.411

In [90]:
#model.most_similar(positive=[u'пожар_NOUN'])
#model.most_similar(positive=[u'пожар_NOUN', u'пламя_NOUN' ])
#model.most_similar(positive=[u'пожар_NOUN', u'пламя_NOUN' ], negative=[u'топливо_NOUN'])
model.most_similar(positive=[u'женщина_NOUN', u'император_NOUN' ], negative=[u'мужчина_NOUN'])
#model.most_similar(positive=[u'женщина_NOUN', u'король_NOUN' ])

d:\IDE\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('императрица_NOUN', 0.54018235206604),
 ('государь_NOUN', 0.4910103678703308),
 ('царь_NOUN', 0.4542006850242615),
 ('самодержец_NOUN', 0.415338397026062),
 ('елизавета::петровна_PROPN', 0.3998357057571411),
 ('империя_NOUN', 0.3950481116771698),
 ('акихито_PROPN', 0.3925309181213379),
 ('мария::федоровна_PROPN', 0.38869190216064453),
 ('анна::иоанновна_PROPN', 0.38844308257102966),
 ('вельможа_NOUN', 0.3760373592376709)]